In [1]:
using ReverseDiff
using ReverseDiff: @forward
using ForwardDiff
using DiffResults

In [2]:
module Nets

abstract type Layer <: Function end

params(f::Function) = ()
with_params(f::Function, params::Tuple{}) = f

params(::Layer) = error("Not implemented")
with_params(::Layer, params) = error("Not implemented")
with_params(::Layer, ::Tuple{}) = error("Not implemented")

struct Affine{TA <: AbstractMatrix, Tb <: AbstractVector} <: Layer
    A::TA
    b::Tb
end
    
params(a::Affine) = (a.A, a.b)
with_params(a::Affine, params) = Affine(params[1], params[2])

(a::Affine)(x) = a.A * x .+ a.b

struct Chain{L <: Tuple{Vararg{Function}}}
    layers::L
end

Chain(layers::Tuple) = Chain{typeof(layers)}(layers)

params(c::Chain) = params.(c.layers)

# todo: unroll
function (c::Chain)(x)
    y = x
    for f in c.layers
        y = f(x)
    end
    y
end

with_params(c::Chain, params::Tuple) = Chain(with_params.(c.layers, params))

end

Nets

In [3]:
c = Nets.Chain((Nets.Affine(fill(1.0, 1, 1), fill(0.5, 1)),))

Nets.Chain{Tuple{Nets.Affine{Array{Float64,2},Array{Float64,1}}}}((Nets.Affine,))

In [4]:
c([1.0])

1-element Array{Float64,1}:
 1.5

In [5]:
Nets.params(c)

(([1.0], [0.5]),)

In [13]:
params = (fill(1.0, 1, 1), [0.25])

([1.0], [0.25])

In [14]:
c2 = Nets.with_params(c, (params,))

Nets.Chain{Tuple{Nets.Affine{Array{Float64,2},Array{Float64,1}}}}((Nets.Affine,))

In [15]:
c2([1.0])

1-element Array{Float64,1}:
 1.25

In [20]:
ReverseDiff.jacobian((p...) -> Nets.with_params(c, (p,))([1.0]), params)

([1.0], [1.0])